In [ ]:
import csv
import numpy as np
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

In [30]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = '1xnMYOD5FkrwVKakAAaAaqOyD'
        consumer_secret = 'hf1ADLyEPWll1rl43sGHdwl7eMWmK1LbzU6ajSxQKbwB86I15D'
        access_token = '732181352326434816-XpBDVRPACxdBtChDFqxVhH2bz5nRGm0'
        access_token_secret = '2tu6wcnVBTy6aZNtNjYrSmzzJv1pAvGI2YbTNgjXPc8rv'
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
 
    def get_tweets(self, query, count = 2000):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
 
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
 
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
 
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))
 
                               
def main():
    # creating object of TwitterClient Class
    screen_name = "TiVo BOLT VOX"
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = 'TiVo BOLT VOX', count = 30000)
 
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    print("Neutral  tweets percentage: {} %".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))
 
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:100]:
        print(tweet['text'])


    with open('%s_Positivetweets.csv' % screen_name, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(["id","created_at","text"])
            writer.writerows(tweet['text'])
    
    pass


    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:100]:
        print(tweet['text'])

'''
    with open('%s_Negativetweets.csv' % screen_name, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(["id","created_at","text"])
            writer.writerows(tweet['text'])
    
    pass
'''

       
if __name__ == "__main__":
    # calling main function
    main()

Positive tweets percentage: 76.47058823529412 %
Negative tweets percentage: 11.764705882352942 %
Neutral  tweets percentage: 11.764705882352942 %


Positive tweets:
TiVo embraces voice controls in new Bolt Vox, Mini Vox, and Vox Remote https://t.co/p0nccH0Esz
New post (TiVo BOLT VOX 3TB, DVR &amp; Streaming Media Player, 4K UHD, Now with Voice Control! (TCD849300V)) has been p… https://t.co/mi5oMzFfmZ
New post (TiVo VOX Remote for TiVo BOLT) has been published on Don Cooper Products - https://t.co/3UmOpHq97O
New post (TiVo BOLT VOX 3TB, DVR &amp; Streaming Media Player, 4K UHD, Now with Voice Control! (TCD849300V)) has been p… https://t.co/dAsxgvc44E
New post (TiVo VOX Remote for TiVo BOLT) has been published on Don Cooper Products - https://t.co/3UmOpHq97O
TiVo’s rumored voice-controlled DVR pops up at Amazon, Best Buy https://t.co/uvlI5Qt6Mz
#TiVo has a lot going on these days. They released the new TiVo #BOLT #VOX and will release… https://t.co/A8WSO2TsPc https://t.co/oJmdwhQhK1
TiV